In [4]:
! pip install pandas plotly ipykernel notebook --quiet
! git pull --rebase


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip
Already up to date.


In [1]:
import datetime
import json
from datetime import datetime, timezone, timedelta

import pandas as pd

In [2]:
# read all files in data folder

import glob

data = []
files = glob.glob('data/*.json')
for file in files:
    with open (file) as f:
        json1 = json.load(f)
        tmp_data = {}
        epoch = file.split("data/")[1].split(".")[0]
        date_time = datetime.fromtimestamp(int(epoch), tz=timezone.utc)
        expiry = (date_time + timedelta(hours=12)).strftime('%d%m%y')
        for j in json1["result"]:
            sym = j["symbol"]
            if expiry in sym and "-BTC-" in sym:
                tmp_data["epoch"] = epoch
                tmp_data["datetime"] = date_time
                tmp_data["expiry"] = expiry
                tmp_data["symbol"] = sym
                tmp_data["spot"] = float(j["spot_price"])
                tmp_data["strike_price"] = j["strike_price"]
                tmp_data["contract_type"] = j["contract_type"]
                if j["quotes"]["best_ask"]:
                    tmp_data["ask"] = float(j["quotes"]["best_ask"])
                if j["quotes"]["best_bid"]:
                    tmp_data["bid"] = float(j["quotes"]["best_bid"])
                
                tmp_data["mark_price"] = float(j["mark_price"])
                tmp_data["delta"] = j["greeks"]["delta"]
                tmp_data["gamma"] = j["greeks"]["gamma"]
                tmp_data["theta"] = j["greeks"]["theta"]
                tmp_data["vega"] = j["greeks"]["vega"]
                data.append(tmp_data)
                tmp_data = {}

In [3]:
df = pd.DataFrame(data)
df.head()

,epoch,datetime,expiry,symbol,spot,strike_price,contract_type,ask,bid,mark_price,delta,gamma,theta,vega
0,1668520543,2022-11-15 13:55:43+00:00,161122,P-BTC-17200-161122,16939.242845,17200,put_options,394.0,377.0,386.126933,-0.6657,0.0006,-114.6372,3.0953
1,1668520543,2022-11-15 13:55:43+00:00,161122,P-BTC-17100-161122,16939.242845,17100,put_options,329.0,320.0,324.615450,-0.6003,0.0007,-123.4764,3.2845
2,1668520543,2022-11-15 13:55:43+00:00,161122,P-BTC-17000-161122,16939.242845,17000,put_options,275.0,267.0,270.685359,-0.5336,0.0007,-129.3423,3.3801
3,1668520543,2022-11-15 13:55:43+00:00,161122,P-BTC-16900-161122,16939.242845,16900,put_options,227.0,219.0,223.446122,-0.4665,0.0007,-131.5534,3.3805
4,1668520543,2022-11-15 13:55:43+00:00,161122,P-BTC-16800-161122,16939.242845,16800,put_options,186.0,178.0,182.981870,-0.4029,0.0006,-130.1654,3.2915


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

for i in range(17000, 18000, 100):
    i = str(i)
    df_plot = df[(df["strike_price"] == i) & (df["contract_type"] == "call_options")]
    fig.add_trace(
        go.Scatter(x=df_plot["datetime"], y=df_plot["mark_price"], name=i+" Call"),
        secondary_y=False,
    )

# fig.add_trace(
#     go.Scatter(x=df["datetime"], y=df["spot"], name="BTC Spot", line=dict(
#         color='black',
#         width=3
#     )),
#     secondary_y=True,
# )

fig.show()